In [ ]:
# TODO Existem varios filtros MP/CA era giro percebermos quais são

In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('budget-data/budget-data-extracted.csv')
df.shape

(6776, 13)

In [2]:
df.isna().sum()

company_name            0
quote_id                0
description             0
qty                    92
unit_price            315
filter_efficiency     228
dimensions             83
item_type             993
Length                245
Height                245
Gutter                245
Depth                5007
Pockets              5007
dtype: int64

In [3]:
cols_to_clean = ['unit_price', 'filter_efficiency','Length','Height','Gutter']
df_clean = df.dropna(subset=cols_to_clean)
df_clean.drop(['qty', 'Depth','Pockets','description','item_type'], axis=1, inplace=True)
df_clean.head()

C:\Users\João\AppData\Local\Temp\ipykernel_1652\2741972995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(['qty', 'Depth','Pockets','description','item_type'], axis=1, inplace=True)


,company_name,quote_id,unit_price,filter_efficiency,dimensions,Length,Height,Gutter
0,A Preditiva,17171.0,7.5,G4,625.500.50,625.0,500.0,50.0
1,A Preditiva,17171.0,27.5,F7,625.500.50,625.0,500.0,50.0
2,A Preditiva,17175.0,28.5,F7,592.490.48,592.0,490.0,48.0
3,A Preditiva,18643.0,27.5,F7,592.592.45,592.0,592.0,45.0
4,A Preditiva,18643.0,17.5,F7,287.592.45,287.0,592.0,45.0


In [5]:
#Find individual names
names = df_clean['company_name'].unique()
proccessed_names = set()
for name in names:
    proccessed_names.add(re.sub(r'[^a-zA-Z]', '', str(name).lower()))

map_names = {}
for i, name in enumerate(proccessed_names):
    map_names[name] = i

In [6]:
df_model = df_clean.copy()
df_model['company_name'] = df_model['company_name'].apply(lambda x: map_names.get(re.sub(r'[^a-zA-Z]', '', str(x).lower())))

filterTypes = ["G1","G2","G3","G4","M5","M6","F7","F8","F9","H13","H14"]

def getFilterType(x):
    for filterType in filterTypes:
        if filterType in x:
            return filterType
    return np.nan

df_model['filter_efficiency'] = df_model['filter_efficiency'].apply(lambda x: getFilterType(x))
df_model = df_model.dropna(subset=['filter_efficiency'])

category_dummies = pd.get_dummies(df_model['filter_efficiency'], prefix='')
df_model.drop(['filter_efficiency','dimensions'], axis=1, inplace=True)

df_model = pd.concat([df_model, category_dummies], axis=1)
df_model.head()
df_model.to_csv('budget-data/budget-data-model.csv', index=False)
df_model.head()

,company_name,quote_id,unit_price,Length,Height,Gutter,_F7,_F8,_F9,_G2,_G3,_G4,_H13,_H14,_M5,_M6
0,0,17171.0,7.5,625.0,500.0,50.0,0,0,0,0,0,1,0,0,0,0
1,0,17171.0,27.5,625.0,500.0,50.0,1,0,0,0,0,0,0,0,0,0
2,0,17175.0,28.5,592.0,490.0,48.0,1,0,0,0,0,0,0,0,0,0
3,0,18643.0,27.5,592.0,592.0,45.0,1,0,0,0,0,0,0,0,0,0
4,0,18643.0,17.5,287.0,592.0,45.0,1,0,0,0,0,0,0,0,0,0


# K Neighbors Regressor

### For the General Model

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

#df_model2 = df_model.drop(['company_name'], axis=1)
df_model2 = df_model.drop(['company_name', 'quote_id'], axis=1)
#df_model2 = df_model.copy()
x_train, x_test, y_train, y_test = train_test_split(df_model2.drop(['unit_price'], axis=1), df_model2['unit_price'], test_size=0.2, random_state=42)
df_model2.head()

,unit_price,Length,Height,Gutter,_F7,_F8,_F9,_G2,_G3,_G4,_H13,_H14,_M5,_M6
0,7.5,625.0,500.0,50.0,0,0,0,0,0,1,0,0,0,0
1,27.5,625.0,500.0,50.0,1,0,0,0,0,0,0,0,0,0
2,28.5,592.0,490.0,48.0,1,0,0,0,0,0,0,0,0,0
3,27.5,592.0,592.0,45.0,1,0,0,0,0,0,0,0,0,0
4,17.5,287.0,592.0,45.0,1,0,0,0,0,0,0,0,0,0


### For the top 3 filters

In [9]:
df_G4 = df_model2[df_model2['_G4'] == 1]
df_F7 = df_model2[df_model2['_F7'] == 1]
df_M5 = df_model2[df_model2['_M5'] == 1]

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import math
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def get_metrics(df, model, scoring_metric):

    x_train, x_test, y_train, y_test = train_test_split(df.drop(['unit_price'], axis=1), df['unit_price'], test_size=0.2, random_state=42)
    grid = GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [2, 3, 5, 7, 10, 15, 17, 20],
                         'weights': ('uniform', 'distance'),
                         'p': [1, 2, 3]},
            cv=5,
            scoring=scoring_metric)

    grid.fit(x_train, y_train)

    y_pred = grid.predict(x_test)

    return {
        'scoring_metric': scoring_metric,
        'model': model,
        'r2': r2_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred),
        'mae': mean_absolute_error(y_test, y_pred),
        'rmse': math.sqrt(mean_squared_error(y_test, y_pred))
    }


metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error", "neg_root_mean_squared_error"]

columns = ['scoring_metric','model','r2', 'mse', 'mae', 'rmse']
df_new = pd.DataFrame(columns=columns)
for metric in metrics:
    for model in [(df_model2, "geral"), (df_G4, "G4"), (df_F7, "F7"), (df_M5, "M5")]:
        df_new = df_new.append(get_metrics(model[0], model[1], metric), ignore_index=True)
df_new.to_csv('results/KNNeighbors.csv', index=False)

In [10]:
kNN = pd.read_csv('results/KNNeighbors.csv')
kNN.sort_values(by=['model'], inplace=True)
kNN.head(kNN.shape[0])

,scoring_metric,model,r2,mse,mae,rmse
2,r2,F7,0.789150,44.327176,3.394644,6.657866
6,neg_mean_squared_error,F7,0.789150,44.327176,3.394644,6.657866
10,neg_mean_absolute_error,F7,0.826502,36.474577,3.299196,6.039419
14,neg_root_mean_squared_error,F7,0.817854,38.292704,3.425816,6.188110
1,r2,G4,0.407117,10.591515,1.426357,3.254461
5,neg_mean_squared_error,G4,0.408916,10.559390,1.421186,3.249522
9,neg_mean_absolute_error,G4,0.399881,10.720785,1.408649,3.274261
13,neg_root_mean_squared_error,G4,0.407117,10.591515,1.426357,3.254461
3,r2,M5,0.634940,12.387928,2.133321,3.519649
7,neg_mean_squared_error,M5,0.634940,12.387928,2.133321,3.519649


# Decision Tree

In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import math
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def get_metrics(df1, model, scoring_metric):

    x_train, x_test, y_train, y_test = train_test_split(df1.drop(['unit_price'], axis=1), df1['unit_price'], test_size=0.2, random_state=42)
    grid = GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ('squared_error', 'friedman_mse', 'absolute_error', 'poisson'),
                         'splitter': ('best', 'random'),
                         'max_features': ('auto', 'sqrt', 'log2')},
            cv=5,
            scoring=scoring_metric)

    grid.fit(x_train, y_train)

    y_pred = grid.predict(x_test)

    return {
        'scoring_metric': scoring_metric,	
        'model': model,
        'r2': r2_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred),
        'mae': mean_absolute_error(y_test, y_pred),
        'rmse': math.sqrt(mean_squared_error(y_test, y_pred))
    }

metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error", "neg_root_mean_squared_error"]

columns = ['scoring_metric','model','r2', 'mse', 'mae', 'rmse']
df_new = pd.DataFrame(columns=columns)
for metric in metrics:
    for model in [(df_model2, "geral"), (df_G4, "G4"), (df_F7, "F7"), (df_M5, "M5")]:
        metrics = get_metrics(model[0], model[1], metric)
        df_new = df_new.append(metrics, ignore_index=True)

df_new.to_csv('results/decisionTree.csv', index=False)

In [11]:
dtree = pd.read_csv('results/decisionTree.csv')
dtree.sort_values(by=['model'], inplace=True)
dtree.head(dtree.shape[0])

,scoring_metric,model,r2,mse,mae,rmse
2,r2,F7,0.885421,24.087987,2.699955,4.907951
6,neg_mean_squared_error,F7,0.805983,40.788321,3.048472,6.386573
10,neg_mean_absolute_error,F7,0.819355,37.977161,3.135326,6.162561
14,neg_root_mean_squared_error,F7,0.846461,32.278741,3.029073,5.681438
1,r2,G4,-0.043345,18.638772,1.552965,4.317264
5,neg_mean_squared_error,G4,0.305595,12.405158,1.481460,3.522096
9,neg_mean_absolute_error,G4,0.206134,14.181963,1.587898,3.765895
13,neg_root_mean_squared_error,G4,-0.303452,23.285433,1.669124,4.825498
3,r2,M5,-0.054725,35.790968,2.834845,5.982555
7,neg_mean_squared_error,M5,0.378010,21.106582,2.550054,4.594190


# Random Forest

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import math
import warnings


def get_metrics(df1, model, scoring_metric):

    x_train, x_test, y_train, y_test = train_test_split(df1.drop(['unit_price'], axis=1), df1['unit_price'], test_size=0.2, random_state=42)
    grid = GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [10, 50, 100],
                         'criterion': ('squared_error', 'absolute_error'),
                         'min_samples_split': [1,2, 3],
                         'max_features': ( 'sqrt', 'log2')},
            cv=5,
            scoring=scoring_metric)

    grid.fit(x_train, y_train)

    y_pred = grid.predict(x_test)

    return {
        'scoring_metric': scoring_metric,	
        'model': model,
        'r2': r2_score(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred),
        'mae': mean_absolute_error(y_test, y_pred),
        'rmse': math.sqrt(mean_squared_error(y_test, y_pred))
    }

metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error", "neg_root_mean_squared_error"]

columns = ['scoring_metric','model','r2', 'mse', 'mae', 'rmse']
df_new = pd.DataFrame(columns=columns)
for metric in metrics:
    for model in [(df_model2, "geral"), (df_G4, "G4"), (df_F7, "F7"), (df_M5, "M5")]:
        metrics = get_metrics(model[0], model[1], metric)
        print(metrics)
        df_new = df_new.append(metrics, ignore_index=True)

df_new.to_csv('results/RandomForest.csv', index=False)

c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\SittySoftware\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "c:\SittySoftware\anaconda3\lib\site-

{'scoring_metric': 'r2', 'model': 'geral', 'r2': 0.6231866855171013, 'mse': 115.40676930789908, 'mae': 2.6671211745244006, 'rmse': 10.742754270106856}


c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\SittySoftware\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "c:\SittySoftware\anaconda3\lib\site-

{'scoring_metric': 'r2', 'model': 'G4', 'r2': 0.5179676047173647, 'mse': 8.611237342301326, 'mae': 1.359844536423841, 'rmse': 2.9344909852138454}


c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\SittySoftware\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "c:\SittySoftware\anaconda3\lib\site-

...

{'scoring_metric': 'neg_root_mean_squared_error', 'model': 'G4', 'r2': 0.5341466278368145, 'mse': 8.322208203572021, 'mae': 1.334505380794702, 'rmse': 2.884823773399689}


c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\SittySoftware\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "c:\SittySoftware\anaconda3\lib\site-

{'scoring_metric': 'neg_root_mean_squared_error', 'model': 'F7', 'r2': 0.793860149897046, 'mse': 43.33696582112759, 'mae': 3.1395988130563794, 'rmse': 6.583081787516208}


c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\SittySoftware\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\SittySoftware\anaconda3\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "c:\SittySoftware\anaconda3\lib\site-

{'scoring_metric': 'neg_root_mean_squared_error', 'model': 'M5', 'r2': 0.6486582663607872, 'mse': 11.922409142268041, 'mae': 2.1259938144329897, 'rmse': 3.4528841773607235}


In [12]:
rf = pd.read_csv('results/RandomForest.csv')
rf.sort_values(by=['model'], inplace=True)
rf.head(rf.shape[0])

,scoring_metric,model,r2,mse,mae,rmse
2,r2,F7,0.858493,29.749237,2.855922,5.454286
6,neg_mean_squared_error,F7,0.862546,28.897176,2.818951,5.375609
10,neg_mean_absolute_error,F7,0.847829,31.990968,2.880533,5.656056
14,neg_root_mean_squared_error,F7,0.793860,43.336966,3.139599,6.583082
1,r2,G4,0.517968,8.611237,1.359845,2.934491
5,neg_mean_squared_error,G4,0.504942,8.843930,1.384298,2.973875
9,neg_mean_absolute_error,G4,0.479194,9.303912,1.392429,3.050231
13,neg_root_mean_squared_error,G4,0.534147,8.322208,1.334505,2.884824
3,r2,M5,0.644652,12.058344,2.229599,3.472513
7,neg_mean_squared_error,M5,0.645555,12.027709,2.129637,3.468099


In [15]:
columns = ['model','filter','scoring_metric','r2', 'mse', 'mae', 'rmse']
results_df = pd.DataFrame(columns=columns)
for model in [(kNN, "KNN"),( dtree, "decisionTree"), (rf, "RandomForest")]:
    for index, row in model[0].iterrows():
        results_df = results_df.append({
            'model': model[1],
            'filter': row['model'],
            'scoring_metric': row['scoring_metric'],
            'r2': row['r2'],
            'mse': row['mse'],
            'mae': row['mae'],
            'rmse': row['rmse']
        }, ignore_index=True)

results_df.to_csv('results/results.csv', index=False)
results_df.head(results_df.shape[0])

C:\Users\João\AppData\Local\Temp\ipykernel_1652\1233751266.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\João\AppData\Local\Temp\ipykernel_1652\1233751266.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\João\AppData\Local\Temp\ipykernel_1652\1233751266.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\João\AppData\Local\Temp\ipykernel_1652\1233751266.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\João\AppData\Local\Temp\ipykernel_1652\1233751266.py:5: Fut

,model,filter,scoring_metric,r2,mse,mae,rmse
0,KNN,F7,r2,0.78915,44.327176,3.394644,6.657866
1,KNN,F7,neg_mean_squared_error,0.78915,44.327176,3.394644,6.657866
2,KNN,F7,neg_mean_absolute_error,0.826502,36.474577,3.299196,6.039419
3,KNN,F7,neg_root_mean_squared_error,0.817854,38.292704,3.425816,6.18811
4,KNN,G4,r2,0.407117,10.591515,1.426357,3.254461
5,KNN,G4,neg_mean_squared_error,0.408916,10.55939,1.421186,3.249522
6,KNN,G4,neg_mean_absolute_error,0.399881,10.720785,1.408649,3.274261
7,KNN,G4,neg_root_mean_squared_error,0.407117,10.591515,1.426357,3.254461
8,KNN,M5,r2,0.63494,12.387928,2.133321,3.519649
9,KNN,M5,neg_mean_squared_error,0.63494,12.387928,2.133321,3.519649


In [18]:
def plot_difference(y_test, y_pred):
    sorted_indices = np.argsort(y_test)

    # Apply sorting to both arrays using the sorted indices
    sorted_y_test = y_test.values[sorted_indices]
    sorted_y_pred = y_pred[sorted_indices]

    plt.figure(figsize=(50,15))
    plt.plot(sorted_y_test, label='Real')
    plt.plot(sorted_y_pred, label='Predicted')
    plt.legend()
    plt.show()

plot_difference(y_test, y_pred)

NameError: name 'y_pred' is not defined

# 